<a href="https://colab.research.google.com/github/niczky12/medium/blob/master/tech/bigquery/BigQuery_UDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
from google.cloud import bigquery
from google.cloud.bigquery import magics
from sklearn.datasets import make_blobs
import pandas as pd

In [2]:
auth.authenticate_user()

In [3]:
PROJECT_ID = "YOURPROJECT"
DATASET_ID = "ds"
magics.context.project = PROJECT_ID

In [4]:
points, labels, centers = make_blobs(
    centers=2,
    n_samples=100,
    n_features=2,
    return_centers=True,
    random_state=12)

In [5]:
df = pd.DataFrame(points, columns=["x", "y"])
df["label"] = labels
df

,x,y,label
0,-10.064160,5.336130,0
1,-5.939345,0.293243,1
2,-4.282810,0.033378,1
3,-3.968270,0.646273,1
4,-5.580912,5.119659,0
...,...,...,...
95,-7.170647,4.874246,0
96,-1.692013,0.048707,1
97,-5.251409,0.278910,1
98,-6.360683,5.023713,0


In [6]:
df.to_gbq("ds.clusters", project_id=PROJECT_ID, if_exists="replace")

1it [00:03,  3.69s/it]


In [7]:
%%bigquery
select *
from ds.clusters

,x,y,label
0,-10.064160,5.336130,0
1,-5.580912,5.119659,0
2,-6.044775,3.354634,0
3,-5.904228,3.887125,0
4,-8.445729,5.830205,0
...,...,...,...
95,-5.674058,0.958395,1
96,-6.572809,0.738567,1
97,-5.217731,-0.838299,1
98,-1.692013,0.048707,1


# What are UDFs?

User defined functions are functions that are not included in BigQuery, but rather defined by you, the user. We can define BigQuery udfs either as SQL or as JavaScript functions.
Let's start by creating a super simple SQL function.

In [8]:
%%bigquery
CREATE TEMP FUNCTION move_x(x FLOAT64, labelid INT64)
  RETURNS FLOAT64
  AS (CASE WHEN labelid = 0 THEN x ELSE x * 100 END);

SELECT
  x
  ,move_x(x, label) AS new_x
  ,y
  ,label
FROM
  ds.clusters

,x,new_x,y,label
0,-10.064160,-10.064160,5.336130,0
1,-5.580912,-5.580912,5.119659,0
2,-6.044775,-6.044775,3.354634,0
3,-5.904228,-5.904228,3.887125,0
4,-8.445729,-8.445729,5.830205,0
...,...,...,...,...
95,-5.674058,-567.405842,0.958395,1
96,-6.572809,-657.280900,0.738567,1
97,-5.217731,-521.773113,-0.838299,1
98,-1.692013,-169.201349,0.048707,1


In [9]:
%%bigquery
select
  x
  ,move_x(x, label) as new_x
  ,y
  ,label
from ds.clusters

Executing query with job ID: f8fe7f26-fe14-40a7-8cc8-12ead4ce3c9e
Query executing: 0.26s


ERROR:
 400 Function not found: move_x at [3:4]

(job ID: f8fe7f26-fe14-40a7-8cc8-12ead4ce3c9e)

 -----Query Job SQL Follows----- 

    |    .    |    .    |
   1:select
   2:  x
   3:  ,move_x(x, label) as new_x
   4:  ,y
   5:  ,label
   6:from ds.clusters
    |    .    |    .    |


In [11]:
%%bigquery
CREATE FUNCTION ds.move_x(x FLOAT64, labelid INT64)
  RETURNS FLOAT64
  AS (CASE WHEN labelid = 0 THEN x ELSE x * 100 END);

""


In [ ]:
%%bigquery
select
  x
  ,ds.move_x(x, label) as new_x
  ,y
  ,label
from ds.clusters

,x,new_x,y,label
0,-10.064160,-10.064160,5.336130,0
1,-5.580912,-5.580912,5.119659,0
2,-6.044775,-6.044775,3.354634,0
3,-5.904228,-5.904228,3.887125,0
4,-8.445729,-8.445729,5.830205,0
...,...,...,...,...
95,-5.674058,-567.405842,0.958395,1
96,-6.572809,-657.280900,0.738567,1
97,-5.217731,-521.773113,-0.838299,1
98,-1.692013,-169.201349,0.048707,1


In [13]:
%%bigquery
CREATE OR REPLACE FUNCTION ds.move_x(x FLOAT64, labelid INT64)
  RETURNS FLOAT64
  AS (CASE WHEN labelid = 0 THEN x ELSE x * 100 END)
OPTIONS (description="Make the x coordinate larger if the cluster id is 1.")

""


In [ ]:
%%bigquery
DROP FUNCTION ds.move_x;

""


Using any types - templating

In [14]:
%%bigquery
CREATE TEMP FUNCTION multiply(x ANY TYPE, mult ANY TYPE)
  AS (x * mult);

select
  x
  ,label
  ,multiply(x, 10) as new_x
  ,multiply(label, 10) as new_label
  ,multiply(label, 1.5) as float_label
from ds.clusters

,x,label,new_x,new_label,float_label
0,-10.064160,0,-100.641597,0,0.0
1,-5.580912,0,-55.809118,0,0.0
2,-6.044775,0,-60.447750,0,0.0
3,-5.904228,0,-59.042277,0,0.0
4,-8.445729,0,-84.457286,0,0.0
...,...,...,...,...,...
95,-5.674058,1,-56.740584,10,1.5
96,-6.572809,1,-65.728090,10,1.5
97,-5.217731,1,-52.177311,10,1.5
98,-1.692013,1,-16.920135,10,1.5


# JS udfs

Especially useful if you need to do stuff with JSON types or arrays. See official docs for a JSON example.


In [15]:
%%bigquery
CREATE TEMP FUNCTION size(x FLOAT64, y FLOAT64)
  RETURNS FLOAT64
  LANGUAGE js AS r"""
return Math.sqrt(Math.pow(x, 2) + Math.pow(y, 2));
""";

select
  x
  ,y
  ,size(x, y) as js_distance
  ,sqrt(x * x + y * y) as sql_distance
from ds.clusters

,x,y,js_distance,sql_distance
0,-10.064160,5.336130,11.391295,11.391295
1,-5.580912,5.119659,7.573473,7.573473
2,-6.044775,3.354634,6.913239,6.913239
3,-5.904228,3.887125,7.068921,7.068921
4,-8.445729,5.830205,10.262632,10.262632
...,...,...,...,...
95,-5.674058,0.958395,5.754430,5.754430
96,-6.572809,0.738567,6.614174,6.614174
97,-5.217731,-0.838299,5.284644,5.284644
98,-1.692013,0.048707,1.692714,1.692714


In [ ]:
%%bigquery
CREATE TEMP FUNCTION size(x FLOAT64, y FLOAT64)
  RETURNS FLOAT64
  LANGUAGE js AS r"""
function euclidean(x, y) {
  return Math.sqrt(Math.pow(x, 2) + Math.pow(y, 2))
}
return euclidean(x, y);
""";

select
  x
  ,y
  ,size(x, y) as js_distance
  ,sqrt(x * x + y * y) as sql_distance
from ds.clusters

,x,y,js_distance,sql_distance
0,-10.064160,5.336130,11.391295,11.391295
1,-5.580912,5.119659,7.573473,7.573473
2,-6.044775,3.354634,6.913239,6.913239
3,-5.904228,3.887125,7.068921,7.068921
4,-8.445729,5.830205,10.262632,10.262632
...,...,...,...,...
95,-5.674058,0.958395,5.754430,5.754430
96,-6.572809,0.738567,6.614174,6.614174
97,-5.217731,-0.838299,5.284644,5.284644
98,-1.692013,0.048707,1.692714,1.692714


In [16]:
%%bigquery
CREATE TEMP FUNCTION size(x FLOAT64, y FLOAT64)
  RETURNS STRING
  LANGUAGE js
  OPTIONS (
    library=["gs://YOURBUCKET/jstat.min.js"]
  )
  AS
r"""
  return Math.sqrt(jStat.sumsqrd([x, y]));
""";

select
  x
  ,y
  ,size(x, y) as js_distance
  ,sqrt(x * x + y * y) as sql_distance
from ds.clusters

,x,y,js_distance,sql_distance
0,-10.064160,5.336130,11.39129454614176,11.391295
1,-5.580912,5.119659,7.573472597458625,7.573473
2,-6.044775,3.354634,6.91323930681272,6.913239
3,-5.904228,3.887125,7.068920959142272,7.068921
4,-8.445729,5.830205,10.262632396220125,10.262632
...,...,...,...,...
95,-5.674058,0.958395,5.754429590034841,5.754430
96,-6.572809,0.738567,6.61417408439208,6.614174
97,-5.217731,-0.838299,5.284644169764053,5.284644
98,-1.692013,0.048707,1.6927143845043258,1.692714


In [17]:
%%bigquery
CREATE TEMP FUNCTION add_noise(id STRING, score FLOAT64)
  RETURNS FLOAT64
  AS (
    (1 - (mod(abs(FARM_FINGERPRINT(id)), 10000) / 10000000)) * score
  );

WITH example as (
    select
        "alice" as name
        ,0.98 as score
    union all
    select
        "bob", 0.7
    union all
    select
        "bob", 0.7
)

select *
    ,add_noise(name, score) as new_score
from example


,name,score,new_score
0,alice,0.98,0.979563
1,bob,0.70,0.699516
2,bob,0.70,0.699516
